# Free-Energy Decomposition of Salt Effects on the Solubilities of Small Molecules and the Role of Excluded-Volume Effects

Stefan Hervø-Hansen<sup>a,&#42;</sup>, Lin Daoyang<sup>a</sup>, Kento Kasahara<sup>a</sup>, and Nobuyuki Matubayasi<sup>a,&#42;</sup>.<br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup>*</sup> To whom correspondence may be addressed: stefan@cheng.es.osaka-u.ac.jp and nobuyuki@cheng.es.osaka-u.ac.jp.

## PART 1: Force field Parameter Generation
For this project we are gonna utilize the [SPC/E](https://doi.org/10.1063/1.2841127) water model, [Heyda _et al_ parameters](https://doi.org/10.1021/jp910953w) for monatomic anions and cations, and finally the solute using the [GAFF-2.11](https://doi.org/10.1002/jcc.20035). While the OPC water model and ions have been manually imported, we desire the option to simulate any arbitrary solute. Consequently this notebook serve exactly as the solution to that problem.

In this notebook we will generate the force field parameters for the solute molecules one choose to study. In order to do so one must include the solute molecule in the variable `solutes` with the `Category` being a subdirectory within the `PDB_files` directory.


## Import of Python Modules & Auxiliary Functions

In [1]:
from openmm.app import PDBFile, ForceField
from openff.toolkit.topology import Molecule, Topology
from openmmforcefields.generators import GAFFTemplateGenerator
import parmed as pmd
import os

homedir = !pwd
homedir = homedir[0]
print(homedir)

/data/group1/z44785r/SI-Cavity-Formation


### Solute list and choice of force field

In [2]:
# Solute list
solutes = {
    'methanol':   {'SMILES': 'CO',      'Category': 'n-alcohols', 'Resname': 'MOH'},
    'ethanol' :   {'SMILES': 'CCO',     'Category': 'n-alcohols', 'Resname': 'EOH'},
    'n-propanol': {'SMILES': 'CCCO',    'Category': 'n-alcohols', 'Resname': 'PHO'},
    'n-butanol' : {'SMILES': 'CCCCO',   'Category': 'n-alcohols', 'Resname': 'BOH'},
    'n-pentanol': {'SMILES': 'CCCCCO',  'Category': 'n-alcohols', 'Resname': 'POH'},
    'n-hexanol':  {'SMILES': 'CCCCCCO', 'Category': 'n-alcohols', 'Resname': 'HHO'},
    
    'methane':    {'SMILES': 'C',       'Category': 'n-alkanes',  'Resname': 'MTH'},
    'ethane' :    {'SMILES': 'CC',      'Category': 'n-alkanes',  'Resname': 'ETH'},
    'propane':    {'SMILES': 'CCC',     'Category': 'n-alkanes',  'Resname': 'PRP'},
    'n-butane' :  {'SMILES': 'CCCC',    'Category': 'n-alkanes',  'Resname': 'BUT'},
    'n-pentane':  {'SMILES': 'CCCCC',   'Category': 'n-alkanes',  'Resname': 'PEN'},
    'n-hexane':   {'SMILES': 'CCCCCC',  'Category': 'n-alkanes',  'Resname': 'HEX'},
}

### Generate parameters

In [3]:
%cd -q $homedir
for solute, solute_properties in solutes.items():
    if not os.path.isfile('Force_fields/{}.itp'.format(solute)): # check if a force field file exist for the solute
        molecule = Molecule.from_smiles(solute_properties['SMILES'])
        gaff = GAFFTemplateGenerator(molecules=molecule, forcefield='gaff-2.11')
        forcefield = ForceField()
        forcefield.registerTemplateGenerator(gaff.generator)
        pdbfile = PDBFile('PDB_files/{}/{}.pdb'.format(solute_properties['Category'], solute))
        system = forcefield.createSystem(pdbfile.topology)
        parmed_structure = pmd.openmm.load_topology(pdbfile.topology, system, pdbfile.positions)
        parmed_structure.save("Force_fields/temporary.top", overwrite=True)        
        with open("Force_fields/temporary.top", "r+") as f:
            lines = f.readlines()
        f.close()
        os.remove('Force_fields/temporary.top')
        
        with open('Force_fields/{}.itp'.format(solute), "w") as f:
            for i,line in enumerate(lines):
                if '[ defaults ]' in line:
                    for j in range(3):
                        lines[i+j] = ''
            f.seek(0)
            for line in lines[:-7]:
                f.write(line)
        f.close()